In [1]:
%pip install fastapi uvicorn pydantic

  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/e6/33/de41e554e5a187d583906e10d53bfae5fd6c07e98cbf4fe5262bd37e739a/fastapi-0.111.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn from https://files.pythonhosted.org/packages/b2/f9/e6f30ba6094733e4f9794fd098ca0543a19b07ac1fa3075d595bf0f1fb60/uvicorn-0.30.1-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic from https://files.pythonhosted.org/packages/6f/b9/ec44b1394957d5aa8d3a7c33f8304cd7670d10a43a286db56cec086346be/pydantic-2.7.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/108.5 kB ? eta -:--:--
     ------------- ----------------------- 41.0/108.5 kB 991.0 kB/s eta 0:00:01
     ----------------------------------- -- 102.4/108.5 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 108.5/108.5 kB 1.0 MB/s eta 0:00:00
  Obtaining dependency information for starlette<0.38.0,>=0.37.2 from https://files.p


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Inteli\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import uvicorn
import threading
import openai

openai.api_key = "sua-chave-de-api-do-openai"


app = FastAPI()

# Modelos Pydantic para validação de dados
class Story(BaseModel):
    title: str
    description: str
    category: str

class User(BaseModel):
    username: str
    email: str

# Banco de dados temporário
stories_db = []
users_db = []

# CRUD de Histórias
@app.post("/stories/")
def create_story(story: Story):
    stories_db.append(story)
    return story

@app.put("/stories/{story_id}")
def update_story(story_id: int, story: Story):
    if story_id >= len(stories_db):
        raise HTTPException(status_code=404, detail="Story not found")
    stories_db[story_id] = story
    return story

@app.delete("/stories/{story_id}")
def delete_story(story_id: int):
    if story_id >= len(stories_db):
        raise HTTPException(status_code=404, detail="Story not found")
    deleted_story = stories_db.pop(story_id)
    return {"message": "Story deleted", "deleted_story": deleted_story}

@app.get("/stories/")
def get_all_stories():
    return stories_db

# Geração de Conteúdo
@app.post("/stories/{story_id}/generate/")
def generate_story_part(story_id: int, prompt: str):
    if story_id >= len(stories_db):
        raise HTTPException(status_code=404, detail="Story not found")
    completion = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=50
    )
    generated_text = completion.choices[0].text.strip()
    stories_db[story_id].description += " " + generated_text
    return {"generated_text": generated_text}

# Gestão de Usuários
@app.post("/users/")
def create_user(user: User):
    users_db.append(user)
    return user

@app.put("/users/{user_id}")
def update_user(user_id: int, user: User):
    if user_id >= len(users_db):
        raise HTTPException(status_code=404, detail="User not found")
    users_db[user_id] = user
    return user

@app.delete("/users/{user_id}")
def delete_user(user_id: int):
    if user_id >= len(users_db):
        raise HTTPException(status_code=404, detail="User not found")
    deleted_user = users_db.pop(user_id)
    return {"message": "User deleted", "deleted_user": deleted_user}

@app.get("/users/")
def get_all_users():
    return users_db

# Documentação interativa (Swagger)
from fastapi.openapi.docs import get_swagger_ui_html

@app.get("/docs", include_in_schema=False)
async def custom_swagger_ui_html():
    return get_swagger_ui_html(openapi_url="/openapi.json", title="API de Histórias Interativas", oauth2_redirect_url="/docs/oauth2-redirect")

@app.get("/openapi.json", include_in_schema=False)
async def get_openapi():
    return app.openapi()

def run_server():
    def run():
        uvicorn.run(app, host="0.0.0.0", port=8080)  # Alterando a porta para 8001

    server_thread = threading.Thread(target=run)
    server_thread.start()

run_server()

INFO:     Started server process [22952]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:51384 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51384 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51384 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51384 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51475 - "GET /stories/ HTTP/1.1" 200 OK
